In [1]:
pip install yelpapi requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from yelpapi import YelpAPI

YELP_API_KEY = 'your-api-key'
yelp_api = YelpAPI(YELP_API_KEY)

In [3]:
cuisine_types = ['chinese', 'italian', 'japanese', 'mexican', 'american']

In [4]:
def fetch_restaurants(cuisine, offset=0):
    return yelp_api.search_query(
        term=f'{cuisine} restaurants',
        location='Manhattan, NY',
        limit=50,
        offset=offset
    )

In [8]:
import time

all_restaurants = []
total_count = 0

try:
    for cuisine in cuisine_types:
        while total_count < 10000:
            try:
                results = fetch_restaurants(cuisine,200)  
                new_restaurants = results['businesses']
                all_restaurants.extend(new_restaurants)
                total_count += len(new_restaurants)
                
                if total_count % 1000 == 0:
                    print(f"Collected {total_count} restaurants. Taking a 5-second break...")
                    time.sleep(5)
                
                if len(new_restaurants) == 0:
                    break  # No more results for this cuisine
                
            except Exception as e:
                print(f"Error fetching restaurants for {cuisine}: {str(e)}")
                break  # Move to next cuisine if there's an error
        
        if total_count >= 10000:
            break

except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")

finally:
    print(f"Total restaurants collected: {len(all_restaurants)}")
    print(f"Total unique restaurants collected: {len(set(restaurant['id'] for restaurant in all_restaurants))}")

Error fetching restaurants for chinese: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?term=chinese+restaurants&location=Manhattan%2C+NY&limit=50&offset=200
Error fetching restaurants for italian: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?term=italian+restaurants&location=Manhattan%2C+NY&limit=50&offset=200
Error fetching restaurants for japanese: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?term=japanese+restaurants&location=Manhattan%2C+NY&limit=50&offset=200
Error fetching restaurants for mexican: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?term=mexican+restaurants&location=Manhattan%2C+NY&limit=50&offset=200
Error fetching restaurants for american: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?term=american+restaurants&location=Manhattan%2C+NY&limit=50&offset=200
Total restaurants collected: 0
Total uniq

In [6]:
unique_restaurants = {restaurant['id']: restaurant for restaurant in all_restaurants}
print(f"Unique restaurants: {len(unique_restaurants)}")

Unique restaurants: 53


In [14]:
# print(unique_restaurants)

In [51]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('restaurants.db')
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS restaurants (
    id TEXT PRIMARY KEY,
    name TEXT,
    address TEXT,
    latitude REAL,
    longitude REAL,
    review_count INTEGER,
    rating REAL,
    zip_code TEXT,
    cuisine TEXT,
    inserted_at INTEGER
)
''')

# Prepare INSERT or UPDATE statement
upsert_query = '''
INSERT INTO restaurants 
(id, name, address, latitude, longitude, review_count, rating, zip_code, cuisine, inserted_at)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
ON CONFLICT(id) DO UPDATE SET
    name = excluded.name,
    address = excluded.address,
    latitude = excluded.latitude,
    longitude = excluded.longitude,
    review_count = excluded.review_count,
    rating = excluded.rating,
    zip_code = excluded.zip_code,
    cuisine = excluded.cuisine,
    inserted_at = excluded.inserted_at
'''

# Insert or update data in the database
new_records = 0
updated_records = 0
for restaurant_id, restaurant in unique_restaurants.items():
    cursor.execute("SELECT id FROM restaurants WHERE id = ?", (restaurant_id,))
    existing = cursor.fetchone()
    
    cursor.execute(upsert_query, (
        restaurant_id,
        restaurant.get('name', ''),
        restaurant.get('location', {}).get('address1', ''),
        restaurant.get('coordinates', {}).get('latitude', None),
        restaurant.get('coordinates', {}).get('longitude', None),
        restaurant.get('review_count', 0),
        restaurant.get('rating', 0.0),
        restaurant.get('location', {}).get('zip_code', ''),
        restaurant.get('categories', [{}])[0].get('alias', ''),
        restaurant.get('inserted_at_timestamp', 0)
    ))
    
    if existing:
        updated_records += 1
    else:
        new_records += 1

# Commit changes and close connection
conn.commit()
conn.close()

print(f"Added {new_records} new restaurants and updated {updated_records} existing records in the database.")

Added 0 new restaurants and updated 0 existing records in the database.


In [50]:
import sqlite3
from collections import defaultdict

# Connect to the SQLite database
conn = sqlite3.connect('restaurants.db')
cursor = conn.cursor()

# Function to display all data
def display_all_data():
    cursor.execute("SELECT * FROM restaurants")
    rows = cursor.fetchall()
    
    print("Total restaurants:", len(rows))
    print("\nSample data (first 5 entries):")
    for row in rows[:5]:
        print(row)

# Function to check for duplicates
def check_duplicates():
    cursor.execute("SELECT id, name, COUNT(*) FROM restaurants GROUP BY id HAVING COUNT(*) > 1")
    duplicates = cursor.fetchall()
    
    if duplicates:
        print("\nDuplicates found:")
        for dup in duplicates:
            print(f"ID: {dup[0]}, Name: {dup[1]}, Count: {dup[2]}")
    else:
        print("\nNo duplicates found based on ID.")

    # Check for potential duplicates based on name and address
    cursor.execute("SELECT name, address, COUNT(*) FROM restaurants GROUP BY name, address HAVING COUNT(*) > 1")
    potential_duplicates = cursor.fetchall()
    
    if potential_duplicates:
        print("\nPotential duplicates (same name and address):")
        for dup in potential_duplicates:
            print(f"Name: {dup[0]}, Address: {dup[1]}, Count: {dup[2]}")
    else:
        print("\nNo potential duplicates found based on name and address.")

# Function to display cuisine distribution
def display_cuisine_distribution():
    cursor.execute("SELECT cuisine, COUNT(*) FROM restaurants GROUP BY cuisine ORDER BY COUNT(*) DESC")
    cuisine_counts = cursor.fetchall()
    
    print("\nCuisine distribution:")
    for cuisine, count in cuisine_counts:
        print(f"{cuisine}: {count}")

# Main execution
if __name__ == "__main__":
    display_all_data()
    check_duplicates()
    display_cuisine_distribution()

    # Close the connection
    conn.close()

Total restaurants: 985

Sample data (first 5 entries):
('dCDpb-fz6Ton9AXLoaWIhA', 'Momoya SoHo', '47 Prince St', 40.723578, -73.995746, 301, 4.1, '10012', 'sushi', 0)
('smxIke5NSkMIExa6jLVw-g', "Joe's Steam Rice Roll", '422 Amsterdam Ave', 40.784079, -73.977885, 255, 4.3, '10024', 'cantonese', 0)
('DynFEEWtMX76qeqviUbOdA', 'Mille Nonne', '88 Essex St', 40.717943, -73.988216, 69, 4.6, '10002', 'italian', 0)
('Zjdtcex73QIz2_MvEvDrFQ', 'MakiMaki Sushi', '1369 6th Ave', 40.7633913312621, -73.9780393019703, 281, 4.1, '10019', 'japanese', 0)
('b3lYtaBR1SlmH7NNDWi-vg', 'The Landing', '1 Pennsylvania Plz', 40.7511917, -73.992165, 39, 4.7, '10119', 'cocktailbars', 0)

No duplicates found based on ID.

No potential duplicates found based on name and address.

Cuisine distribution:
italian: 170
mexican: 130
japanese: 89
newamerican: 74
chinese: 70
sushi: 58
szechuan: 36
cocktailbars: 35
dimsum: 30
ramen: 27
bars: 22
tradamerican: 17
pizza: 16
tacos: 15
shanghainese: 14
breakfast_brunch: 13
french